# BACQUET Maxime et CARON Mathieu

# TP3 - *Latent Dirichlet Allocation* et Inférence variationnelle 

## Estimation avancée - G3 SDIA

Dans ce TP, on s'intéresse à la méthode "inférence variationnelle" (VI) qui permet d'approcher la loi a posteriori d'un modèle (généralement inconnue) par une autre loi plus simple (généralement un produit de lois bien connues). Nous allons l'appliquer à un modèle probabiliste pour des données textuelles, appelé *Latent Dirichlet Allocation* (LDA, qui n'a rien à voir avec la LDA *Linear Discriminant Analysis* du cours de ML).

### Instructions

1. Renommer votre notebook sous la forme `tp3_Nom1_Nom2.ipynb`, et inclure le nom du binôme dans le notebook. 

2. Votre code, ainsi que toute sortie du code, doivent être commentés !

3. Déposer votre notebook sur Moodle dans la section prévue à cet effet avant la date limite : 23 Décembre 2023, 23h59.

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pickle as pkl

### Partie 0 - Introduction

LDA is a popular probabilistic model for text data, introducted in [Blei et al. (2003)](https://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf). In this model, the posterior distribution is intractable, and we choose to resort to variational inference (note that a Gibbs sampler would be feasible as well, but would be very slow). In particular, the CAVI updates can be easily derived.

In a few words, in LDA, each document is a mixture of topics, and each topic is a mixture of words. Uncovering those is the goal of *topic modeling*, and this is what we are going to do today. We will be using a collection of abstracts of papers published in JMLR (*Journal of Machine Learning Research*), one of the most prominent journals of the field.

**Check the .pdf file describing the model.**
The posterior is :
$$p(\boldsymbol{\beta}, \boldsymbol{\theta}, \mathbf{z} | \mathcal{D}),$$
which we are going to approximate in the following way :
$$\simeq \left[ \prod_{k=1}^K q(\beta_k) \right] \left[ \prod_{d=1}^D q(\theta_d) \right] \left[ \prod_{d=1}^D \prod_{n=1}^{N_d} q(z_{dn}) \right], $$
with :
* $q(\beta_k)$ a Dirichlet distribution (of size V) with parameter $[\lambda_{k1}, ...,\lambda_{kV}]$
* $q(\gamma_d)$ a Dirichlet distribution (of size K) with parameter $[\gamma_{d1}, ...,\gamma_{dK}]$
* $q(z_{dn})$ a Multinomial distribution (of size K) with parameter $[\phi_{dn1}, ..., \phi_{dnK}]$

The updates are as follows :
* $$\lambda_{kv} = \eta + \sum_{d=1}^D \sum_{n=1}^{N_d} w_{dnv} \phi_{dnk} $$
* $$\gamma_{dk} = \alpha + \sum_{n=1}^{N_d} \phi_{dnk}$$
* $$ \phi_{dnk} \propto \exp \left( \Psi(\gamma_{dk}) + \Psi(\lambda_{k, w_{dn}}) - \Psi(\sum_{v=1}^V \lambda_{kv}) \right)$$

$\Psi$ is the digamma function, use `scipy.special.digamma`.

### Partie 1 - Les données

The data is already prepared, see code below. We have a total of 1898 abstracts.

In [2]:
jmlr_papers = pkl.load(open("jmlr.pkl","rb"))

**Q1.** Fill in a list of keywords from the course, to see how many papers are about probabilistic ML.

In [3]:
bayesian_jmlr_papers = []

for paper in jmlr_papers:
    bayesian_keywords = ['bayesian', 'probabilistic', 'Markov', 'mcmc', 'Gibbs sampling', 'posterior', 'prior', 
                         'likelihood', 'Monte Carlo', 'variational inference']
    if any([kwd in paper["abstract"] for kwd in bayesian_keywords]):
        bayesian_jmlr_papers.append(paper)
        
print("There are", str(len(bayesian_jmlr_papers))+" Bayesian papers out of", str(len(jmlr_papers)))

There are 501 Bayesian papers out of 1898


Let us now preprocess the data. It is important to remove so-called "stop-words" like a, is, but, the, of, have... Scikit-learn will do the job for us. We will keep only the top-1000 words from the abstracts.

As a result, we get the count matrix $\mathbf{C}$ of size $D = 1898 \times V = 1000$. $c_{dv}$ is the number of occurrences of word $v$ in document $d$. This compact representation is called "bag-of-words". Of course from $\mathbf{C}$ you easily recover the words, since in LDA the order does not matter.

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features = 1000, stop_words='english')
X = vectorizer.fit_transform([paper["abstract"] for paper in jmlr_papers])
print(vectorizer.get_feature_names_out()) # Top-1000 words
C = X.toarray() # Count matrix

# Removing documents with 0 words
idx = np.where(np.sum(C, axis = 1)==0)
C = np.delete(C, idx, axis = 0)

AttributeError: 'CountVectorizer' object has no attribute 'get_feature_names_out'

**Q2.** How many elements of $\mathbf{C}$ are non-zero ? Is this surprising ?

In [5]:
number_total_entries = len(C) * len(C[0])
print("Le nombre total d'entrées dans la matrice C est :", number_total_entries)

num_nonzero_elements = np.count_nonzero(C)
print("Le nombre d'éléments non nuls dans la matrice C est :", num_nonzero_elements)

print(f"La proportion d'éléments non nuls dans cette matrice est donc : {num_nonzero_elements / number_total_entries * 100:.3f}%")

NameError: name 'C' is not defined

Ces résultats montrent que sur les 1895 documents et 1000 mots possibles, seuls 85 804 de ces éléments sont différents de zéro. La représentation du "bag-of-words" est donc creuse pour des ensembles de données textuelles (on a une matrice avec beaucoup de 0)

Ceci n'est pas surprenant car chaque document utilise généralement seulement un petit sous-ensemble de l'ensemble du vocabulaire, conduisant à une matrice où la majorité des entrées sont nulles.

### Partie 2 - Inférence variationnelle

As you know from the lecture, VI aims at maximizing the ELBO. I have prepared for you the function to compute the ELBO.

In [28]:
from scipy.special import digamma, loggamma

def ELBO(L, G, phi, a, e, W):
    # Computes the ELBO with the values of the parameters L (Lambda), G (Gamma), and Phi
    # a, e are hyperparameters (alpha and eta)
    # W are the words (obsereved)
    
    # L - K x V matrix (variational parameters Lambda)
    # G - D x K matrix (variational parameters Gamma)
    # phi - List of D elements, each element is a Nd x K matrix (variational parameters Phi)
    # a - Scalar > 0 (hyperparameter alpha)
    # e - Scalar > 0 (hyperparameter eta)
    # W - List of D elements, each element is a Nd x V matrix (observed words)
    (K, V) = L.shape
    D = len(phi)
    e_log_B = (digamma(L).T - digamma(np.sum(L, axis = 1))).T
    e_log_T = (digamma(G).T - digamma(np.sum(G, axis = 1))).T
    
    t1 = (e-1)*np.sum(e_log_B)
    t2 = (a-1)*np.sum(e_log_T)

    phi_s = np.zeros((D,K))
    for d in range(0,D):
        phi_s[d,:] = np.sum(phi[d], axis = 0)
    t3 = np.sum(e_log_T*phi_s)
    
    tmp = np.zeros((K,V))
    for d in range(0,D):
        tmp = tmp + np.dot(phi[d].T, W[d])
    t4 = np.sum(e_log_B*tmp)
    
    t5 = np.sum(loggamma(np.sum(L, axis = 1))) - np.sum(loggamma(L)) + np.sum((L-1)*e_log_B)
    t6 = np.sum(loggamma(np.sum(G, axis = 1))) - np.sum(loggamma(G)) + np.sum((G-1)*e_log_T)

    t7 = 0
    for d in range(0,D):
        t7 = t7 + np.sum(phi[d]*np.log(phi[d] + np.spacing(1)))

    return t1 + t2 + t3 + t4 - t5 - t6 - t7

**Q1.** Transform the matrix $\mathbf{C}$ into the observed words $\mathbf{w}$. $\mathbf{w}$ should be a list of $D$ elements, each element of the list being a $N_d \times V$ matrix.

In [25]:
D, V = C.shape

# Liste pour stocker les mots observés
w = []

for doc in range(D):
    # On extrait la ligne des counts
    counts_d = C[doc, :]
    n_d = sum(counts_d)
    
    # Create a matrix with counts for each word in the vocabulary
    matrix_d = np.zeros((n_d, V))

    i = 0
    k = 0
    for v in range(len(counts_d)):
        if counts_d[v] != 0:
            for j in range(counts_d[v]):
                matrix_d[i, k] = 1
                i += 1
            k += 1
    
    w.append(matrix_d)

**Q2.** Implement the CAVI algorithm. The updates are given at the beginning of the notebook. Monitor the convergence with the values of the ELBO (but start with a fixed number of iterations, like 50).

which we are going to approximate in the following way :
$$\simeq \left[ \prod_{k=1}^K q(\beta_k) \right] \left[ \prod_{d=1}^D q(\theta_d) \right] \left[ \prod_{d=1}^D \prod_{n=1}^{N_d} q(z_{dn}) \right], $$
with :
* $q(\beta_k)$ a Dirichlet distribution (of size V) with parameter $[\lambda_{k1}, ...,\lambda_{kV}]$
* $q(\gamma_d)$ a Dirichlet distribution (of size K) with parameter $[\gamma_{d1}, ...,\gamma_{dK}]$
* $q(z_{dn})$ a Multinomial distribution (of size K) with parameter $[\phi_{dn1}, ..., \phi_{dnK}]$

The updates are as follows :
* $$\lambda_{kv} = \eta + \sum_{d=1}^D \sum_{n=1}^{N_d} w_{dnv} \phi_{dnk} $$
* $$\gamma_{dk} = \alpha + \sum_{n=1}^{N_d} \phi_{dnk}$$
* $$ \phi_{dnk} \propto \exp \left( \Psi(\gamma_{dk}) + \Psi(\lambda_{k, w_{dn}}) - \Psi(\sum_{v=1}^V \lambda_{kv}) \right)$$

$\Psi$ is the digamma function, use `scipy.special.digamma`.

In [29]:
def CAVI(W, K, a, e, seed, max_iters=50, tol=1e-5):
    """
    Coordinate Ascent Variational Inference (CAVI) for Latent Dirichlet Allocation (LDA).

    Parameters:
    - W (list): List of D elements, each element is a Nd x V matrix (observed words).
    - K (int): Number of topics.
    - a (float): Hyperparameter for Gamma distribution (document-topic distribution).
    - e (float): Hyperparameter for Gamma distribution (word-topic distribution).
    - seed (int): Seed for random number generation.
    - max_iters (int): Maximum number of iterations.
    - tol (float): Convergence tolerance.
    """
    np.random.seed(seed)
    
    # Nombre de documents
    D = len(W)
    # Taille du vocabulaire
    V = W[0].shape[1]

    # Initialisation des paramètres variationnels
    L = np.random.rand(K, V)
    G = np.random.rand(D, K)
    phi = [np.zeros((Wd.shape[0], K)) for Wd in W]

    for iter in range(max_iters):
        # Mise à jour phi
        for d in range(D):
            for n in range(len(phi[d])):
                # Indice où il y a le 1 dans la matrice W[d][n, :]
                w_dn = np.where(W[d][n, :] == 1)[0][0]
                # Calcul de phi_dn pour tous les thèmes en une seule opération
                phi[d][n, :] = np.exp(digamma(G[d, :]) + digamma(L[:, w_dn]) - digamma(np.sum(L, axis=1)))
                # Normalisation
                phi[d][n, :] /= np.sum(phi[d][n, :])

        # Calcul de l'ELBO et vérification de la convergence
        current_ELBO = ELBO(L, G, phi, a, e, W)
        print(f"Iteration {iter + 1}, ELBO: {current_ELBO}")

        # Vérification de la convergence
        if iter > 0 and np.abs((current_ELBO - prev_ELBO) / prev_ELBO) < tol:
            print(f"Converged after {iter + 1} iterations.")
            break

        # Mise à jour Gamma
        for d in range(D):
            G[d, :] = a + np.sum(phi[d], axis=0)

        # Mise à jour Lambda
        for v in range(V):
            L[:, v] = e + np.sum(np.dot(W[d][:, v].T, phi[d]) for d in range(D))

        prev_ELBO = current_ELBO
    
    return L, G, phi

**Q3.** Run the algorithm with $K = 10$, $\alpha = 0.5$, $\eta = 0.1$. From the results, compute the MMSE of $\lambda_{kv}$ and $\gamma_{dk}$.

**Bonus** : Re-run the algorithm several times with different initializations, and keep the solution which returns the highest ELBO.

NB : In my implementation, one iteration of the CAVI algorithm takes about 4 seconds to run.

In [30]:
L, G, phi = CAVI(w, 10, 0.5, 0.1, 42)

Iteration 1, ELBO: -1126928.1606178735


C:\Users\maxim\AppData\Local\Temp\ipykernel_11820\3454808762.py:52: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  L[:, v] = e + np.sum(np.dot(W[d][:, v].T, phi[d]) for d in range(D))


Iteration 2, ELBO: -496327.67318333546
Iteration 3, ELBO: -494222.5006113121
Iteration 4, ELBO: -492829.6389252198
Iteration 5, ELBO: -491640.3938148209
Iteration 6, ELBO: -490408.0329103642
Iteration 7, ELBO: -488969.75641519914
Iteration 8, ELBO: -487240.5776350163
Iteration 9, ELBO: -485244.84176680376
Iteration 10, ELBO: -483142.07131999114
Iteration 11, ELBO: -481144.70761278813
Iteration 12, ELBO: -479385.1651698362
Iteration 13, ELBO: -477880.122661098
Iteration 14, ELBO: -476601.225351761
Iteration 15, ELBO: -475521.43694675574
Iteration 16, ELBO: -474621.3897803674
Iteration 17, ELBO: -473876.16825919354
Iteration 18, ELBO: -473262.95783701085
Iteration 19, ELBO: -472753.0917168412
Iteration 20, ELBO: -472315.0009172772
Iteration 21, ELBO: -471928.2366804841
Iteration 22, ELBO: -471576.3462595348
Iteration 23, ELBO: -471250.16884724435
Iteration 24, ELBO: -470945.9563372721
Iteration 25, ELBO: -470660.42326672934
Iteration 26, ELBO: -470391.5960322425
Iteration 27, ELBO: -4701

In [43]:
L_MMSE = np.argmax(L)
G_MMSE = np.argmax(G)
print(L_MMSE, G_MMSE)

2029 9064


AttributeError: 'CountVectorizer' object has no attribute 'shape'

In [45]:
'''init = [(10, 0.5, 0.1)]
elbo_max = 0
K_max, a_max, e_max = 0, 0, 0
for (K, a, e) in init:
    L, G, phi = CAVI(w, K, a, e, 42)
    elbo = ELBO(L, G, phi, a, e, w)
    if elbo > elbo_max:
        elbo_max = elbo
        K_max, a_max, e_max = K, a, e
print(f"Les paramètres d'initialisation pour lesquelles la ELBO est maximale sont K = {K_max}, alpha = {a_max}, eta = {e_max}")'''

Iteration 1, ELBO: -1126928.1606178735


C:\Users\mathc\AppData\Local\Temp\ipykernel_30696\3454808762.py:52: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  L[:, v] = e + np.sum(np.dot(W[d][:, v].T, phi[d]) for d in range(D))


Iteration 2, ELBO: -496327.67318333546
Iteration 3, ELBO: -494222.5006113121
Iteration 4, ELBO: -492829.63892522006


KeyboardInterrupt: 

**Q4.** Based on the MMSE estimates :
* What are the top-10 words per topic ? With your machine learning knowledge, can you make sense of some of the topics ?
* Choose one document at random and display its topic proportions. Comment.

In [ ]:
#######
### YOUR CODE HERE
#######

----- Your answer here -----

**Q5.** Open questions :
* What are some limitations of the LDA model ? Can you imagine an improvement ?
* In this notebook, we have treated the hyperparameters as fixed. How could they be learned ?
* Can you imagine a method to choose the number of topics ?
* What strategies should we use to make the algorithm more efficient ?

# Limitations du modèle LDA et Améliorations Potentielles :

## Limitations :

1. **Supposition du Nombre Fixe de Thèmes :** LDA suppose un nombre fixe de thèmes dans tout le corpus, ce qui peut ne pas refléter avec précision la structure sous-jacente de différentes parties du corpus.

2. **Supposition de Mélanges de Thèmes Fixes :** LDA suppose que les mélanges de thèmes sont constants au sein d'un document. Cependant, en réalité, la distribution des thèmes au sein d'un document peut varier.

3. **Représentation de Sacs de Mots :** LDA utilise une représentation de sacs de mots, ignorant l'ordre des mots dans les documents, ce qui peut entraîner une perte d'informations sémantiques importantes.

4. **Difficulté à Gérer la Polysémie et la Synonymie :** LDA a du mal avec les mots ayant plusieurs significations (polysémie) ou différents mots exprimant des significations similaires (synonymie).

## Améliorations Potentielles :

1. **Modèles de Thèmes Dynamiques :** Intégrer le temps en tant que facteur peut améliorer LDA pour des applications où les thèmes évoluent au fil du temps.

2. **Incorporation de Mots :** Intégrer des incorporations de mots pour capturer les relations sémantiques entre les mots et leurs contextes peut améliorer la compréhension des significations des mots par le modèle.

3. **Modèles de Thèmes Corrélés :** Assouplir l'hypothèse d'indépendance des thèmes pour permettre des corrélations entre les thèmes peut conduire à des modèles plus réalistes.

# Apprentissage des Hyperparamètres :

Les hyperparamètres du modèle LDA, tels que les priors de Dirichlet (α et β), peuvent être appris à partir des données. Quelques approches comprennent :

1. **Bayes Empirique :** Estimer les hyperparamètres en fonction des données en maximisant la vraisemblance marginale.

2. **Validation Croisée :** Utiliser des techniques de validation croisée pour trouver les valeurs des hyperparamètres qui conduisent à une meilleure généralisation du modèle.

# Choix du Nombre de Thèmes :

Choisir le bon nombre de thèmes est un aspect crucial de LDA. Plusieurs méthodes peuvent être utilisées :

1. **Perplexité :** Utiliser la perplexité sur un ensemble de données détenu pour évaluer le modèle avec différents nombres de thèmes et choisir celui qui minimise la perplexité.

2. **Cohérence des Thèmes :** Évaluer la cohérence des thèmes à l'aide de métriques comme C_v ou la cohérence UMass pour trouver un nombre optimal de thèmes.

3. **Modèles Hiérarchiques :** Explorer des modèles hiérarchiques de thèmes qui déterminent automatiquement le nombre de thèmes à différents niveaux.

# Rendre l'Algorithme Plus Efficace :

1. **Inférence Variationnelle en Mini-Batch :** Implémenter une descente de gradient stochastique en mini-batch pour accélérer le processus d'optimisation, particulièrement sur de grands ensembles de données.

2. **Parallélisation :** Utiliser le traitement parallèle pour distribuer les calculs sur plusieurs cœurs ou nœuds, en particulier lors des mises à jour de l'étape E.

3. **Bibliothèques Optimisées :** Utiliser des bibliothèques ou des frameworks optimisés, tels que des implémentations parallélisées dans des bibliothèques populaires comme scikit-learn ou des frameworks de calcul distribué comme Spark.

4. **Inférence Approximative :** Explorer des méthodes d'inférence plus évolutives, telles que l'échantillonnage de Gibbs effondré ou l'inférence variationnelle en ligne.

5. **Élagage du Modèle :** Supprimer les mots peu fréquents ou non informatifs pour réduire la taille du vocabulaire et accélérer les calculs.

6. **Surveillance de la Convergence :** Mettre en œuvre des mécanismes de surveillance de la convergence pour arrêter le processus d'optimisation lorsque le modèle a convergé, évitant des itérations inutiles.

7. **Représentations Sparser :** Exploiter la sparsité des matrices document-thème et thème-mot pour optimiser le stockage et les calculs.


**BONUS.** Papier-crayon. À partir du modèle, pouvez-vous dériver les lois conditionnelles de l'échantillonneur de Gibbs ? Pour rappel, nous avons besoin de ces lois pour dériver ensuite les updates de l'algorithme CAVI.